In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import cm as CM
from matplotlib.colors import LogNorm,PowerNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry import *
from scipy.interpolate import griddata
import cmocean


In [2]:
fp = "D:/data/map/prd_9city.shp" #生成的坐标不对
mapfile = gpd.read_file(fp,encoding='gbk')

In [3]:
mapfile

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,VARNAME_2,NL_NAME_2,HASC_2,CC_2,TYPE_2,ENGTYPE_2,VALIDFR_2,VALIDTO_2,REMARKS_2,Shape_Leng,Shape_Area,geometry
0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,6.049119,0.630133,(POLYGON ((113.6270840000001 22.72736200000008...
1,45,CHN,China,576,Guangdong,9811,Huizhou,Hu锟絲hou,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,9.867862,0.997726,(POLYGON ((114.8334730000001 22.51375000000002...
2,45,CHN,China,576,Guangdong,9812,Jiangmen,Jiangm锟絥,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,10.246350,0.822387,"(POLYGON ((112.810143 21.58236100000005, 112.8..."
3,45,CHN,China,576,Guangdong,9807,Dongguan,Dongguan,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,2.794603,0.212934,"POLYGON ((113.7502300000001 23.13159400000012,..."
4,45,CHN,China,576,Guangdong,9808,Foshan,F锟絪han,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,4.464386,0.342003,"POLYGON ((112.877854 23.56911800000012, 112.88..."
5,45,CHN,China,576,Guangdong,9820,Shenzhen,Shenzh锟絥,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,5.365538,0.168399,(POLYGON ((114.6665270000001 22.40847199999996...
6,45,CHN,China,576,Guangdong,9824,Zhaoqing,Zh锟給q锟絥g,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,6.746065,1.314538,"POLYGON ((112.30918 24.37709799999999, 112.318..."
7,45,CHN,China,576,Guangdong,9825,Zhongshan,Zhongshan,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,2.062425,0.154312,"POLYGON ((113.4164960000001 22.739847, 113.423..."
8,45,CHN,China,576,Guangdong,9826,Zhuhai,Zhuhai,???,None,None,D锟絡锟絪h?,Prefecture City,Unknown,Present,None,9.575177,0.131368,(POLYGON ((113.9743050000001 21.82319500000011...


In [4]:
data = pd.read_csv('C:/Users/xigrug/Desktop/CCN/new/all_out.csv', sep=',',skiprows=0,header=0)

In [5]:
data

,lat_bin,lon_bin,BJC,B4LI,CAPE,CAPE_nc,CCN_1.0_cm3,CCN_1.0_mg,CIN0,CIN180,...,mean_wspd,max_wspd,max_wdir,ext_wspd,ssd,mean_prs,max_prs,min_prs,mean_rh,min_rh
0,26.0,119.0,2013/6/1 5:09,-6.900,2822.0,1904.0,1337.000000,1316.000000,0.000000,-2.000000,...,14.500000,43.000000,7.500000,59.500000,75.500000,9989.0,10021.00000,9954.00000,80.500000,62.000000
1,28.0,119.5,2013/6/1 5:09,-4.900,1670.0,1551.0,890.000000,986.000000,-1.000000,-2.000000,...,3.000000,40.000000,13.000000,144.000000,0.000000,9982.0,10010.00000,9951.00000,76.000000,52.000000
2,30.5,120.0,2013/6/1 5:09,4.700,20.0,17.0,120.000000,140.000000,0.000000,0.000000,...,14.000000,40.000000,2.000000,65.000000,0.000000,10058.0,10071.00000,10047.00000,89.000000,79.000000
3,31.0,120.0,2013/6/1 5:09,4.700,20.0,11.0,98.000000,124.000000,0.000000,0.000000,...,9.000000,29.000000,8.500000,58.500000,0.000000,10053.5,10066.00000,10041.00000,88.000000,74.000000
4,31.0,121.0,2013/6/1 5:09,4.100,27.0,20.0,128.000000,155.000000,0.000000,0.000000,...,15.000000,28.500000,1.500000,49.500000,0.000000,10058.5,10074.50000,10047.00000,84.000000,61.500000
5,34.0,117.0,2013/6/1 5:09,0.800,95.0,59.0,1393.000000,1514.000000,-88.000000,-105.000000,...,17.000000,35.000000,5.000000,61.000000,43.000000,10013.0,10022.00000,9995.00000,80.000000,51.000000
6,36.0,120.0,2013/6/1 5:09,3.500,0.0,0.0,812.666667,688.666667,0.000000,0.000000,...,23.000000,38.500000,8.000000,69.000000,59.500000,10001.5,10012.50000,9977.50000,84.000000,60.000000
7,32.0,119.0,2013/6/5 5:36,-2.500,1403.0,1326.0,1007.000000,950.000000,0.000000,-1.000000,...,34.000000,54.500000,7.500000,97.000000,79.000000,10099.5,10119.00000,10079.50000,71.000000,48.500000
8,28.0,119.5,2013/6/6 5:13,-1.700,528.0,979.0,573.000000,598.000000,-16.000000,-32.000000,...,8.000000,25.000000,4.000000,55.000000,54.000000,10035.0,10051.00000,10003.00000,67.000000,49.000000
9,31.0,121.0,2013/6/6 5:13,-0.800,617.0,586.0,549.000000,578.000000,-19.000000,-29.000000,...,42.500000,60.500000,6.000000,94.500000,39.500000,10107.0,10130.00000,10081.50000,72.500000,60.500000


In [6]:
data['Coordinates'] = list(zip(data.lon_bin, data.lat_bin))
data['Coordinates'] = data['Coordinates'].apply(Point)

In [7]:
# get geo data
gdf = gpd.GeoDataFrame(data, geometry='Coordinates')
gdf.crs = mapfile.crs
#gdf=gdf.dropna(subset=[ '2001'])#Remember to reassign
cities_with_country = gpd.sjoin(mapfile,gdf,how="inner", op='intersects')
#print(df1)
#df1.to_csv("cities_with_country.csv")

In [8]:
gdf

,lat_bin,lon_bin,BJC,B4LI,CAPE,CAPE_nc,CCN_1.0_cm3,CCN_1.0_mg,CIN0,CIN180,...,max_wspd,max_wdir,ext_wspd,ssd,mean_prs,max_prs,min_prs,mean_rh,min_rh,Coordinates
0,26.0,119.0,2013/6/1 5:09,-6.900,2822.0,1904.0,1337.000000,1316.000000,0.000000,-2.000000,...,43.000000,7.500000,59.500000,75.500000,9989.0,10021.00000,9954.00000,80.500000,62.000000,POINT (119 26)
1,28.0,119.5,2013/6/1 5:09,-4.900,1670.0,1551.0,890.000000,986.000000,-1.000000,-2.000000,...,40.000000,13.000000,144.000000,0.000000,9982.0,10010.00000,9951.00000,76.000000,52.000000,POINT (119.5 28)
2,30.5,120.0,2013/6/1 5:09,4.700,20.0,17.0,120.000000,140.000000,0.000000,0.000000,...,40.000000,2.000000,65.000000,0.000000,10058.0,10071.00000,10047.00000,89.000000,79.000000,POINT (120 30.5)
3,31.0,120.0,2013/6/1 5:09,4.700,20.0,11.0,98.000000,124.000000,0.000000,0.000000,...,29.000000,8.500000,58.500000,0.000000,10053.5,10066.00000,10041.00000,88.000000,74.000000,POINT (120 31)
4,31.0,121.0,2013/6/1 5:09,4.100,27.0,20.0,128.000000,155.000000,0.000000,0.000000,...,28.500000,1.500000,49.500000,0.000000,10058.5,10074.50000,10047.00000,84.000000,61.500000,POINT (121 31)
5,34.0,117.0,2013/6/1 5:09,0.800,95.0,59.0,1393.000000,1514.000000,-88.000000,-105.000000,...,35.000000,5.000000,61.000000,43.000000,10013.0,10022.00000,9995.00000,80.000000,51.000000,POINT (117 34)
6,36.0,120.0,2013/6/1 5:09,3.500,0.0,0.0,812.666667,688.666667,0.000000,0.000000,...,38.500000,8.000000,69.000000,59.500000,10001.5,10012.50000,9977.50000,84.000000,60.000000,POINT (120 36)
7,32.0,119.0,2013/6/5 5:36,-2.500,1403.0,1326.0,1007.000000,950.000000,0.000000,-1.000000,...,54.500000,7.500000,97.000000,79.000000,10099.5,10119.00000,10079.50000,71.000000,48.500000,POINT (119 32)
8,28.0,119.5,2013/6/6 5:13,-1.700,528.0,979.0,573.000000,598.000000,-16.000000,-32.000000,...,25.000000,4.000000,55.000000,54.000000,10035.0,10051.00000,10003.00000,67.000000,49.000000,POINT (119.5 28)
9,31.0,121.0,2013/6/6 5:13,-0.800,617.0,586.0,549.000000,578.000000,-19.000000,-29.000000,...,60.500000,6.000000,94.500000,39.500000,10107.0,10130.00000,10081.50000,72.500000,60.500000,POINT (121 31)


In [9]:
cities_with_country = gpd.sjoin(mapfile,gdf,how="inner", op='intersects')

In [10]:
cities_with_country=cities_with_country.drop(["geometry"],axis=1)

In [11]:
cities_with_country.to_csv("prd_ccn_new-all.csv")

In [12]:
df=pd.read_csv("prd_ccn_new-all.csv")

In [13]:
df.head()

,Unnamed: 0,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,VARNAME_2,NL_NAME_2,...,mean_wspd,max_wspd,max_wdir,ext_wspd,ssd,mean_prs,max_prs,min_prs,mean_rh,min_rh
0,0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,...,21.0,65.0,12.0,120.0,98.0,10039.0,10051.0,10020.0,81.0,57.0
1,0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,...,26.0,71.0,11.0,116.0,72.0,10055.0,10067.0,10041.0,81.0,58.0
2,0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,...,16.0,50.0,10.0,96.0,68.0,10022.0,10039.0,10003.0,81.0,59.0
3,0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,...,27.0,50.0,9.0,93.0,111.0,10037.0,10048.0,10012.0,77.0,56.0
4,0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,...,10.0,38.0,3.0,62.0,69.0,9998.0,10016.0,9971.0,85.0,56.0


In [14]:
df=df.drop(["Unnamed: 0"],axis=1)

In [15]:
df

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,VARNAME_2,NL_NAME_2,HASC_2,...,mean_wspd,max_wspd,max_wdir,ext_wspd,ssd,mean_prs,max_prs,min_prs,mean_rh,min_rh
0,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,21.0,65.0,12.0,120.0,98.0,10039.0,10051.0,10020.0,81.0,57.0
1,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,26.0,71.0,11.0,116.0,72.0,10055.0,10067.0,10041.0,81.0,58.0
2,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,16.0,50.0,10.0,96.0,68.0,10022.0,10039.0,10003.0,81.0,59.0
3,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,27.0,50.0,9.0,93.0,111.0,10037.0,10048.0,10012.0,77.0,56.0
4,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,10.0,38.0,3.0,62.0,69.0,9998.0,10016.0,9971.0,85.0,56.0
5,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,12.0,46.0,7.0,63.0,108.0,10030.0,10048.0,10003.0,78.0,52.0
6,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,15.0,24.0,2.0,39.0,51.0,10049.0,10073.0,10022.0,87.0,59.0
7,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,28.0,57.0,15.0,103.0,13.0,9993.0,10011.0,9973.0,94.0,75.0
8,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,12.0,96.0,3.0,135.0,92.0,10019.0,10037.0,9993.0,79.0,50.0
9,45,CHN,China,576,Guangdong,9809,Guangzhou,Guangzhou,???,NaN,...,33.0,77.0,4.0,129.0,19.0,9932.0,9958.0,9907.0,81.0,69.0


In [19]:
df["DATE"]

0       2013/7/2
1       2013/7/7
2      2013/6/26
3      2015/6/27
4      2016/8/11
5      2017/6/11
6       2016/8/5
7      2014/6/19
8       2015/8/6
9      2016/8/16
10      2015/7/2
11     2013/8/19
12     2016/6/29
13     2016/6/13
14     2014/7/10
15      2013/8/3
16     2017/7/18
17     2016/8/21
18     2017/8/30
19     2017/8/24
20     2017/8/11
21     2016/6/24
22      2016/7/7
23     2016/7/15
24      2014/7/5
25     2013/7/18
26     2017/8/24
27     2017/6/11
28      2015/7/8
29      2017/8/3
         ...    
73     2017/6/11
74     2016/6/29
75     2014/7/10
76      2015/7/2
77      2016/8/5
78     2016/8/21
79     2013/7/18
80      2015/7/8
81     2016/8/11
82      2013/7/7
83     2015/6/27
84     2016/7/15
85      2014/7/5
86     2015/7/29
87     2016/6/24
88     2013/8/19
89     2013/6/26
90     2014/7/10
91      2015/7/2
92     2015/6/11
93     2016/6/13
94      2016/8/5
95     2015/7/29
96     2016/7/15
97     2015/6/27
98     2016/8/21
99     2013/7/23
100    2017/8/

In [20]:
avg_data=df.groupby(['DATE']).mean()

In [21]:
avg_data.to_csv("prd_ccn_avg_new-all.csv")